## Borrar tablas de raw según el subtype del catálogo

Este código sirve para eliminar las tablas de catálogo que incluyan el subtype del catálogo a importar. De esta manera, solo estamos borrando las tablas que posiblemente se vayan a importar en esta ejecución sin tocar las que se pueden estar generando en otro proceso (evitamos problemas de concurrencia) o eliminar aquellas que todavía no se vayan a importar y perder su trazabilidad.

En el caso de Logisfashion, todos los datos que se importan desde Logiscore (BBDD origen de la mayoria de sus datos) se importan con el prefijo "sqllogiscore_" para todas las tablas. Dentro de estas tablas, hay diferentes subtypes de SQL:
- SQL
- SQLNS
- SQLREAL ...

El objetivo es mantener la legibilidad de las tablas y el buen funcionamiento de los borrados.

In [1]:
def get_unique_tablelist(dataset_tables):
    """
    Devuelve una lista con los nombres únicos de las tablas del dataset.

    Parametros:
    dataset_tables (pandas dataframe): El resultado de la query de datasets de la BBDD catálogo

    Devuelve:
    final_tablenames (lista de strings): Lista con los nombres únicos de las tablas de los datasets a tratar
    
    """
    
    landing_tables = list(dataset_tables.groupby("TableName").groups.items())

    # Crear un conjunto para almacenar los nombres únicos
    final_tablenames = set()

    # Procesar cada entrada en la lista
    for table_name, _ in landing_tables:

        final_tablenames.add(table_name.lower())

    # Convertir el conjunto a una lista (opcional)
    final_tablenames = list(final_tablenames)

    # Imprimir los nombres únicos
    print(final_tablenames)

    return final_tablenames

In [2]:
def delete_tables(schema, tablename_list, table_prefix = ""):
    """
    Elimina las tablas del esquema y prefijo que se le pasa por parametros

    Parametros:
    schema (string): Catalogo + Esquema de donde se quieren borrar los datos
    tablename_list (lista de strings): Lista con los nombres únicos de las tablas de los datasets a tratar
    table_prefix (string): prefijo de los datos. Si no tiene, el valor por defecto es un string vacío

    Devuelve:
    Nada
    """
    
    # Obtener la lista de tablas en el esquema
    tables = spark.catalog.listTables(schema)
    tables = [table for table in tables if table_prefix in table.name]

    # Este paso es opcional segun tus datos. En Logisfashion los nombres de las tablas son catalogo.esquema.sqllogiscore_inbounds_esp <prefijo_tabla_origenpais>.
    # Con el siguiente código nos estamos quedando solo con el nombre de la tabla.
    tables = [table for table in tables if table.name.split("_")[-2] in tablename_list]

    # Iterar sobre la lista y eliminar cada tabla
    try:
        for table in tables:
            table_name = table.name
            print("Eliminando ", schema_name, ".", table_name)
            spark.sql(f"DROP TABLE {schema_name}.{table_name}")
    except Exception as e:
        print(str(e))

In [ ]:
# Define el nombre del esquema
schema_name = "pro.raw"

# Define el prefijo de los datos
table_prefix = "sqllogiscore_"


table_list = get_unique_tablelist(sql_datasets)

delete_tables(schema_name, table_list, table_prefix)